In [ ]:
#!pip install prettytable

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import cv2
import os
import numpy as np
from prettytable import PrettyTable

In [9]:
def load_images_from_folder(folder):
    '''loads the data for a  given directory '''
    images ={} 
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename),0)
        if img is not None:
            images[filename]=img
            
    return images


In [13]:
def MED(image):
    ''' for a given  image apply the   MED(Median Edge Detector) predictor '''
    a, b = image.shape
    temp = np.zeros((a,b))
    temp[0,:] = image[0,:]
    temp[:,b-1] = image[:,b-1]
    for i in range(1,a):
        for j in range(1,b-1):
            w = image[i,j-1]
            nw = image[i-1,j-1]
            n = image[i-1, j]
            ne = image[i-1,j+1]
            max = np.maximum(w,n)
            min = np.minimum(w,n)

            if nw >= max:
                temp[i,j] = min
            elif nw <= min:
                temp[i,j] = max
            else:
                temp[i,j] = w+n-nw
    return temp

In [14]:
def GAP(image):
    ''' for a given  image apply the   GAP(Gradient Adjusted Predictor) predictor ''' 
    a, b = image.shape
    temp = np.zeros((a,b))
    temp[0:2,:] = image[0:2,:]
    temp[:,b-2:b] = image[:,b-2:b]
    for i in range (2,a):
        for j in range (2,b-2):
            w = image[i,j-1]
            ww = image[i,j-2]
            n = image[i-1,j]
            nw = image[i-1,j-1]
            ne = image[i-1,j+1]
            nn = image[i-2,j]
            nne = image[i-2,j+2]

            dh = np.abs(w - ww) + np.abs(n - nw) + np.abs(n - ne)
            dv = np.abs(w - nw) + np.abs(n - nn) + np.abs(ne - nne)
            d = dv - dh
            if (d>80):
                temp[i,j] = w
            elif (d<-80):
                temp[i,j] = n
            else:
                temp[i,j] = (w+n)/2 + (ne-nw)/4
                if(d>32):
                    temp[i,j] = (temp[i,j] + w)/2
                elif(d>8):
                    temp[i,j] = (3 * temp[i,j] + w)/4
                elif(d<-32):
                    temp[i,j] = (temp[i,j] + n)/2
                elif(d<-8):
                    temp[i,j] = (3 * temp[i,j] + n)/4
    return temp

In [5]:
def JPEG_Old_v_1(image):
    ''' for a given  image apply the   JPEG_Old_v_1) predictor ''' 
    a, b = image.shape
    temp = np.zeros((a,b))
    temp[0,:] = image[0,:]
    for n in range(1,a):
        temp[n,:] = image[n-1,:]
    return temp  

In [4]:
def JPEG_Old_v_6(image):
    ''' for a given  image apply the JPEG_Old_v_6 predictor ''' 
    a, b = image.shape
    temp = np.zeros((a,b))
    temp[0,:] = image[0,:]
    temp[:,0] = image[:,0]
    for i in range(1,a):
        for j in range(1,b):
            temp[i,j] = image[i-1,j] + (image[i,j-1] - image[i-1,j-1]) / 2
    return temp  

In [10]:
def Calc_Errore (image1 , image2):
    ''' calculates the difference between image 1 and image 2 => image1 - image 2 ''' 
    #if (shape1 != image1.shape) != (shape2 != image2.shape):
     #   raise Exception(f"domension do not match image 1 shape = {shape1}  \n image2 shape = {shape2}") 
    return np.mean(image1-image2)

In [11]:
def print_results(path):
    '''for a directory applies 4 predictor and calculates the diffs '''
    files=load_images_from_folder(path)
    t = PrettyTable(['file_name',"MED" ,"GAP","JPEG_Old_v_1","JPEG_Old_v_6"])
    fucntions=[MED ,GAP ,JPEG_Old_v_1 ,JPEG_Old_v_6 ]
    for filename,image in files.items():
        diffs=[]
        diffs.append(filename)
        for func in fucntions:
            tmp=func(image)
            tmp=Calc_Errore(image,tmp)
            diffs.append(tmp)
        t.add_row(diffs)

    return t 

In [15]:
diff=print_results(r'/content/drive/MyDrive/Colab Notebooks/distorted images/')
print (diff)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in ubyte_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in ubyte_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in ubyte_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in ubyte_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: overflow encountered in ubyte_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in ubyte_scalars
  if __name__ == '__main__':


+--------------+--------------------+--------------------+-----------------------+---------------------+
|  file_name   |        MED         |        GAP         |      JPEG_Old_v_1     |     JPEG_Old_v_6    |
+--------------+--------------------+--------------------+-----------------------+---------------------+
| I01_01_1.bmp | 0.1955718994140625 | 6.9812266031901045 |   -0.03887939453125   |  -61.53676096598307 |
| I01_01_4.bmp | 0.1230010986328125 | 5.062659581502278  | -0.038324991861979164 | -62.592447916666664 |
| I01_03_1.bmp | 0.208221435546875  | 6.693362236022949  |   -0.03668212890625   |  -61.04466247558594 |
| I01_03_4.bmp | 0.1635284423828125 | 5.430887540181478  | -0.034917195638020836 |  -62.02108256022135 |
| I01_07_1.bmp | 0.3329874674479167 | 10.161924362182617 | -0.020350138346354168 |  -36.64653778076172 |
| I01_07_4.bmp | 0.8397725423177084 | 24.602691650390625 |  -0.10828145345052083 |  -22.68779754638672 |
+--------------+--------------------+------------------

in jpeg v1 the mean error between two image is low and that means adjacent pixels are similar and its good for jpeg v1 for compression.

As can be seen when noises are increased, in noise 1 and 3 errors are reduced and good for compression but in noise 7 with increasing noise, pixels are less similar each other and compression is high error with this med and gap method but because of existing noise adjacent pixel are more similar and it is better to using jpeg method.

generally, existing noise causes that prediction with adjacent pixel is harder than usual but when increasing noise, compression is easiest than before because noisy pixels are similar to each other